* DCGAN使ってグラタンに似た偽画像を生成する

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# カレントディレクトリの読み込みとカレントディレクトリへの移動
import sys
sys.path.append(f'/content/drive/My Drive/system/')
import os
os.chdir(f'/content/drive/My Drive/system/myanswer')

In [3]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-7v2x3pzi
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-7v2x3pzi
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=6e7b9adc8388c981c27c414c03045f08147530a8488d9b2e97f7ef8723dccc2b
  Stored in directory: /tmp/pip-ephem-wheel-cache-n_mqebw5/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [4]:
!pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 103kB/s 
ERROR: umap-learn 0.4.6 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [5]:
from __future__ import print_function, division
from keras.datasets import mnist
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from glob import glob
from skimage.transform import resize 
import datetime
import pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.misc
import imageio

In [6]:
class DataLoader():
    def __init__(self, dataset_name, img_res=(128, 128)):
        self.dataset_name = dataset_name
        self.img_res = img_res

    def load_data(self, is_testing=False):
        if os.path.exists("../pickle/{}_tensor.pickle".format(self.dataset_name)):
            with open("../pickle/{}_tensor.pickle".format(self.dataset_name), 'rb') as p:
                imgs = pickle.load(p)
        else:
            img_pathes = glob('../figure/foodimg128/%s/*.jpg' % (self.dataset_name))
            imgs = []
            for img_path in img_pathes:
                img = self.imread(img_path)
                if not is_testing:
                    img = scipy.misc.imresize(img, self.img_res)
                    if np.random.random() > 0.5:
                        img = np.fliplr(img)
                else:
                    img = scipy.misc.imresize(img, self.img_res)
                imgs.append(img)
            with open('../pickle/{}_tensor.pickle'.format(self.dataset_name), 'wb') as p:
                pickle.dump(imgs , p)

        return np.array(imgs)

    def imread(self, path):
        return scipy.misc.imread(path, mode="RGB").astype(np.float)

In [7]:
class DCGAN():
    def __init__(self, dataset_name="mnist"):
        # Input shape
        self.img_rows = 28
        self.img_cols = 28
        # 変換させたい画像のデータセットの名前を指定
        self.dataset_name = dataset_name
        if self.dataset_name == "mnist":
            self.channels = 1
        else:
            self.channels = 3

        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)
      

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, valid)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()
        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1, activation='sigmoid'))

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, save_interval=50):

        if self.dataset_name == "mnist":
            (X_train, _), (_, _) = mnist.load_data()
            # Rescale -1 to 1
            X_train = X_train / 127.5 - 1.
            X_train = np.expand_dims(X_train, axis=3)
        else:
            X_train = self.data_loader.load_data()
            # Rescale -1 to 1
            X_train = X_train / 127.5 - 1.


        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random half of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]
          
            # Sample noise and generate a batch of new images
            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
            gen_imgs = self.generator.predict(noise)
        

            # Train the discriminator (real classified as ones and generated as zeros)
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            # Train the generator (wants discriminator to mistake images as real)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print("epoch %d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch+1, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % save_interval == 0:
                self.save_imgs(epoch)

    def save_imgs(self, epoch):
        os.makedirs('../result/%s/dcgan' % self.dataset_name, exist_ok=True)
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                if self.dataset_name == "mnist":
                    axs[i,j].imshow(gen_imgs[cnt,:,:,0], cmap="grey")
                else:
                    axs[i,j].imshow(gen_imgs[cnt,:,:,:])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("../result/{}/dcgan/epoch{}.png".format(self.dataset_name, epoch),
                     transparent=True, dpi=300, bbox_inches="tight", pad_inches=0.0)
        plt.close()

In [8]:
dcgan = DCGAN(dataset_name="gratin")
# dcgan = DCGAN(dataset_name="mnist")
dcgan.train(epochs=4000, batch_size=32, save_interval=50)

epoch 1 [D loss: 1.179046, acc.: 28.12%] [G loss: 0.650335]
epoch 2 [D loss: 1.050846, acc.: 48.44%] [G loss: 0.678777]
epoch 3 [D loss: 0.599279, acc.: 70.31%] [G loss: 0.698485]
epoch 4 [D loss: 0.590291, acc.: 70.31%] [G loss: 0.744585]
epoch 5 [D loss: 0.342577, acc.: 90.62%] [G loss: 0.798785]
epoch 6 [D loss: 0.343241, acc.: 90.62%] [G loss: 0.702493]
epoch 7 [D loss: 0.313051, acc.: 89.06%] [G loss: 0.653982]
epoch 8 [D loss: 0.374288, acc.: 79.69%] [G loss: 0.509993]
epoch 9 [D loss: 0.181334, acc.: 95.31%] [G loss: 0.389137]
epoch 10 [D loss: 0.203184, acc.: 93.75%] [G loss: 0.143702]
epoch 11 [D loss: 0.276874, acc.: 90.62%] [G loss: 0.132701]
epoch 12 [D loss: 0.448235, acc.: 79.69%] [G loss: 0.242257]
epoch 13 [D loss: 0.327403, acc.: 85.94%] [G loss: 0.429843]
epoch 14 [D loss: 0.521884, acc.: 76.56%] [G loss: 0.834284]
epoch 15 [D loss: 0.566220, acc.: 78.12%] [G loss: 0.761042]
epoch 16 [D loss: 0.454690, acc.: 73.44%] [G loss: 1.371912]
epoch 17 [D loss: 0.378479, acc.: